In [371]:
# Задание на повторение материала предыдущего семестра

In [1]:
# Зависимости
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.cluster import KMeans

from sklearn.metrics import mean_squared_error, f1_score, silhouette_score

In [2]:
# Генерируем уникальный seed
my_code = "Galeev"
seed_limit = 2 ** 32
my_seed = int.from_bytes(my_code.encode(), "little") % seed_limit

In [3]:
# Данные загружены отсюда: https://www.kaggle.com/dwdkills/russian-demography
# Читаем данные из файла
example_data = pd.read_csv("datasets/russian_demography.csv")

In [4]:
# "year" - год (1990-2017)
# "region" - название региона
# "npg" - естественный прирост населения на 1000 человек
# "birth_rate" - количество рождений на 1000 человек
# "death_rate" - количество смертей на 1000 человек
# "gdw" - коэффициент демографической нагрузки на 100 человек (Отношение числа нетрудоспособных к числу трудоспособных).
# "urbanization" - процент городского населения

example_data.head()

,year,region,npg,birth_rate,death_rate,gdw,urbanization
0,1990,Republic of Adygea,1.9,14.2,12.3,84.66,52.42
1,1990,Altai Krai,1.8,12.9,11.1,80.24,58.07
2,1990,Amur Oblast,7.6,16.2,8.6,69.55,68.37
3,1990,Arkhangelsk Oblast,3.7,13.5,9.8,73.26,73.63
4,1990,Astrakhan Oblast,4.7,15.1,10.4,77.05,68.01


In [5]:
# Так как список регионов меняется от года к году, в данных есть строки без значений. Удалим их
example_data.dropna(inplace=True)

In [6]:
# Определим размер валидационной и тестовой выборок
val_test_size = round(0.2*len(example_data))
print(val_test_size)

463


In [7]:
# Создадим обучающую, валидационную и тестовую выборки
random_state = my_seed
train_val, test = train_test_split(example_data, test_size=val_test_size, random_state=random_state)
train, val = train_test_split(train_val, test_size=val_test_size, random_state=random_state)
print(len(train), len(val), len(test))

1389 463 463


In [8]:
# Значения в числовых столбцах преобразуем к отрезку [0,1].
# Для настройки скалировщика используем только обучающую выборку.
columns_to_scale = ['year', 'npg', 'birth_rate', 'death_rate', 'gdw', 'urbanization']

ct = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), columns_to_scale)], remainder='passthrough')
ct.fit(train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['year', 'npg', 'birth_rate', 'death_rate',
                                  'gdw', 'urbanization'])])

In [9]:
# Преобразуем значения, тип данных приводим к DataFrame
sc_train = pd.DataFrame(ct.transform(train))
sc_test = pd.DataFrame(ct.transform(test))
sc_val = pd.DataFrame(ct.transform(val))

In [10]:
# Устанавливаем названия столбцов
column_names = columns_to_scale + ['region']
sc_train.columns = column_names
sc_test.columns = column_names
sc_val.columns = column_names

In [11]:
sc_train

,year,npg,birth_rate,death_rate,gdw,urbanization,region
0,0.62963,0.497512,0.401709,0.37037,0.056125,0.541705,Chukotka Autonomous Okrug
1,0.333333,0.246269,0.119658,0.555556,0.675413,0.426869,Kurgan Oblast
2,0.296296,0.587065,0.269231,0.083333,0.137967,0.782113,Yamalo-Nenets Autonomous Okrug
3,1.0,0.39801,0.252137,0.416667,0.609209,0.701453,Krasnoyarsk Krai
4,0.296296,0.263682,0.064103,0.462963,0.508601,0.7135,Kaliningrad Oblast
...,...,...,...,...,...,...,...
1384,0.074074,0.328358,0.132479,0.416667,0.734318,0.523242,Penza Oblast
1385,0.0,0.579602,0.444444,0.287037,0.720765,0.5967,Republic of North Ossetia-Alania
1386,0.111111,0.350746,0.179487,0.425926,0.779844,0.466152,Chuvash Republic
1387,0.925926,0.335821,0.213675,0.490741,0.611121,0.566584,Belgorod Oblast


In [12]:
# Вспоминаем алгоритмы решения задачи регрессии: линейную регрессию и метод k ближайших соседей
r_models = []

# Линейная регрессия
# Для использования регуляризации, вместо LinearRegression используем Lasso, Ridge или ElasticNet
# Параметр alpha - коэффициент регуляризации для Lasso и Ridge, по умолчанию равен 1
# Для ElasticNet, если регуляризация иммет вид a*L1+b*L2, то
# параметр alpha = a + b, по умолчанию равен 1
# параметр l1_ratio = a / (a + b), по умолчанию равен 0.5
r_models.append(LinearRegression())
r_models.append(Lasso(alpha=1.0))
r_models.append(Ridge(alpha=1.0))
r_models.append(ElasticNet(alpha=1.0, l1_ratio=0.5))

# K ближайших соседей
# Параметр n_neighbors - количество соседей, по умолчания равен 5
r_models.append(KNeighborsRegressor(n_neighbors=5))
r_models.append(KNeighborsRegressor(n_neighbors=10))
r_models.append(KNeighborsRegressor(n_neighbors=15))

In [13]:
# Выделим предикторы и зависимую переменную
x_labels = column_names[0:-2]
y_labels = ['urbanization']

x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = sc_train[y_labels]
y_test = sc_test[y_labels]
y_val = sc_val[y_labels]

In [14]:
# Обучаем модели
for model in r_models:
    model.fit(x_train, y_train)

In [15]:
# Оценииваем качество работы моделей на валидационной выборке.
mses = []
for model in r_models:
    val_pred = model.predict(x_val)
    mse = mean_squared_error(y_val, val_pred)
    mses.append(mse)
    print(mse)

0.017690323055487036
0.02651012424384232
0.017582079553005153
0.02651012424384232
0.01014656371377071
0.01087842898198293
0.011323085857275228


In [16]:
# Выбираем лучшую модель
i_min = mses.index(min(mses))
best_r_model = r_models[i_min]
best_r_model

KNeighborsRegressor()

In [17]:
# Вычислим ошибку лучшей модели на тестовой выборке.
test_pred = best_r_model.predict(x_test)
mse = mean_squared_error(y_test, test_pred)
print(mse)

0.011668010386334993


In [18]:
# Вспоминаем алгоритмы решения задачи классификации:
# логистическую регрессию, наивный байесовский классификатор и (снова) метод k ближайших соседей
c_models = []

# Логистическая регрессия
# Параметр penalty - тип регуляризации: 'l1', 'l2', 'elasticnet', 'none'}, по умолчанию 'l2'
# Для некоторых типов регуляризации доступны не все алгоритмы (параметр solver)
# Для elasticnet регуляризации необходимо уазывать параметр l1_ratio (0 - l2, 1 - l1)
c_models.append(LogisticRegression(penalty='none', solver='saga'))
c_models.append(LogisticRegression(penalty='l1', solver='saga'))
c_models.append(LogisticRegression(penalty='l2', solver='saga'))
c_models.append(LogisticRegression(penalty='elasticnet', l1_ratio=0.5, solver='saga'))
c_models.append(LogisticRegression())

# Наивный байесовский классификатор
# Параметр alpha - параметр сглаживания, по умолчанию равен 1 (сглаживание Лапласа)
c_models.append(MultinomialNB(alpha=0.0))
c_models.append(MultinomialNB(alpha=0.5))
c_models.append(MultinomialNB(alpha=1.0))

# K ближайших соседей
# Параметр n_neighbors - количество соседей, по умолчания равен 5
c_models.append(KNeighborsClassifier(n_neighbors=5))
c_models.append(KNeighborsClassifier(n_neighbors=10))
c_models.append(KNeighborsClassifier(n_neighbors=15))

In [19]:
# Выделим предикторы и метки классов
x_labels = column_names[0:-1]
y_labels = ['region']

x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = np.ravel(sc_train[y_labels])
y_test = np.ravel(sc_test[y_labels])
y_val = np.ravel(sc_val[y_labels])

In [20]:
# Обучаем модели
for model in c_models:
    model.fit(x_train, y_train)

/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/naive_bayes.py:509: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


In [21]:
# Оценииваем качество работы моделей на валидационной выборке.
f1s = []
for model in c_models:
    val_pred = model.predict(x_val)
    f1 = f1_score(y_val, val_pred, average='weighted')
    f1s.append(f1)
    print(f1)

0.2759077897570022
0.12888921511681356
0.06740238123901787
0.06947184121974041
0.06740238123901787
0.021970564769246134
0.016568563208926468
0.009395594506285649
0.3204683453956485
0.21396909609366221
0.21264766316308642


In [22]:
# Выбираем лучшую модель
i_min = f1s.index(min(f1s))
best_c_model = c_models[i_min]
best_c_model

MultinomialNB()

In [23]:
# Вычислим ошибку лучшей модели на тестовой выборке.
test_pred = best_c_model.predict(x_test)
f1 = f1_score(y_test, test_pred, average='weighted')
print(f1)

0.02131677989330307


In [24]:
# Вспоминаем алгоритм решения задачи кластеризации - метод k-средних
# Параметр n_clusters - количество кластеров, по умолчанию равен 8
k_models = []
k_models.append(KMeans(n_clusters=5))
k_models.append(KMeans(n_clusters=8))
k_models.append(KMeans(n_clusters=20))
k_models.append(KMeans(n_clusters=50))

In [25]:
# Выделим используемые параметры
x_labels = column_names[0:-1]
x = pd.concat([sc_train[x_labels], sc_val[x_labels], sc_test[x_labels]])
x

,year,npg,birth_rate,death_rate,gdw,urbanization
0,0.62963,0.497512,0.401709,0.37037,0.056125,0.541705
1,0.333333,0.246269,0.119658,0.555556,0.675413,0.426869
2,0.296296,0.587065,0.269231,0.083333,0.137967,0.782113
3,1.0,0.39801,0.252137,0.416667,0.609209,0.701453
4,0.296296,0.263682,0.064103,0.462963,0.508601,0.7135
...,...,...,...,...,...,...
458,0.481481,0.594527,0.320513,0.12963,0.069157,0.782244
459,0.62963,0.179104,0.102564,0.652778,0.484622,0.451355
460,0.962963,0.524876,0.423077,0.365741,0.66099,0.46314
461,0.407407,0.131841,0.051282,0.694444,0.668983,0.496923


In [26]:
# Произведем кластеризацию
for model in k_models:
    model.fit(x)

In [27]:
# Оценим качество результата
sils = []
for model in k_models:
    cluster_labels = model.predict(x)
    s = silhouette_score(x, cluster_labels)
    sils.append(s)
    print(s)

0.2952637480262442
0.25440316380375794
0.23903674498647037
0.2330892451084149


In [28]:
# Выбираем лучшую модель
i_min = sils.index(min(sils))
best_k_model = k_models[i_min]
print(best_k_model)
print(sils[i_min])



KMeans(n_clusters=50)
0.2330892451084149


In [29]:
# Задание №1 - анализ моделей для задачи регрессии
# Общий список моделей
r_models = [
    LinearRegression(),
    Lasso(alpha=1.0),
    Lasso(alpha=0.5),
    Ridge(alpha=1.0),
    Ridge(alpha=0.5),
    ElasticNet(alpha=1.0, l1_ratio=0.5),
    ElasticNet(alpha=1.0, l1_ratio=0.25),
    ElasticNet(alpha=1.0, l1_ratio=0.75),
    ElasticNet(alpha=0.5, l1_ratio=0.5),
    ElasticNet(alpha=0.5, l1_ratio=0.25),
    ElasticNet(alpha=0.5, l1_ratio=0.75),
    KNeighborsRegressor(n_neighbors=5),
    KNeighborsRegressor(n_neighbors=10),
    KNeighborsRegressor(n_neighbors=15),
    KNeighborsRegressor(n_neighbors=20),
    KNeighborsRegressor(n_neighbors=25)
]

In [30]:
# Выбор моделей для задания
n = 4
random.seed(my_seed)
my_models1 = random.sample(r_models, n)
print(my_models1)

[Lasso(), KNeighborsRegressor(), KNeighborsRegressor(n_neighbors=25), Ridge(alpha=0.5)]


In [31]:
# Загрузим данные для задачи регрессии
data = pd.read_csv("datasets/weather.csv")
data

,water_level,precipitation,temperature,humidity,visibility,wind,weather,pressure,fire,wl_change,temp_change,pressure_change
0,"468,0","0,0","24,9","31,0","100,0","4,0","0,0","993,8","4,0","-3,0","-0,3","-0,4"
1,"140,0","0,0","27,9","22,0","20,0","1,0","0,0","1004,1","4,0","-2,0","0,6","-1,0"
2,"464,0","0,0","25,6","28,0","20,0","2,0","1,0","1001,2","4,0","-7,0","-1,8","-0,3"
3,"467,0","0,0","26,7","26,0","100,0","3,0","0,0","992,3","4,0","-1,0","1,2","-1,3"
4,"138,0","0,0","29,1","22,0","20,0","1,0","0,0","1001,3","4,0","-2,0","2,5","-1,3"
...,...,...,...,...,...,...,...,...,...,...,...,...
1434,"474,0","0,1","6,4","95,0","4,0","1,0","3,0","999,9","4,0","-1,0","5,9","1,5"
1435,"65,0","0,0","6,4","93,0","10,0","4,0","3,0","983,9","4,0","-1,0","5,2","3,1"
1436,"159,0","0,1","8,5","83,0","10,0","2,0","1,0","969,7","4,0","1,0","1,4","1,9"
1437,"99,0","0,1","7,5","91,0","10,0","3,0","3,0","995,4","4,0","-4,0","2,7","1,7"


In [32]:
data['water_level'] = data['water_level'].str.replace(',','.')
data['precipitation'] = data['precipitation'].str.replace(',','.')
data['temperature'] = data['temperature'].str.replace(',','.')
data['humidity'] = data['humidity'].str.replace(',','.')
data['visibility'] = data['visibility'].str.replace(',','.')
data['wind'] = data['wind'].str.replace(',','.')
data['weather'] = data['weather'].str.replace(',','.')
data['pressure'] = data['pressure'].str.replace(',','.')
data['fire'] = data['fire'].str.replace(',','.')                       
data['wl_changer'] = data['wl_change'].str.replace(',','.')                       
data['temp_change'] = data['temp_change'].str.replace(',','.')                      
data['pressure_change'] = data['pressure_change'].str.replace(',','.')                         

In [33]:
data

,water_level,precipitation,temperature,humidity,visibility,wind,weather,pressure,fire,wl_change,temp_change,pressure_change,wl_changer
0,468.0,0.0,24.9,31.0,100.0,4.0,0.0,993.8,4.0,"-3,0",-0.3,-0.4,-3.0
1,140.0,0.0,27.9,22.0,20.0,1.0,0.0,1004.1,4.0,"-2,0",0.6,-1.0,-2.0
2,464.0,0.0,25.6,28.0,20.0,2.0,1.0,1001.2,4.0,"-7,0",-1.8,-0.3,-7.0
3,467.0,0.0,26.7,26.0,100.0,3.0,0.0,992.3,4.0,"-1,0",1.2,-1.3,-1.0
4,138.0,0.0,29.1,22.0,20.0,1.0,0.0,1001.3,4.0,"-2,0",2.5,-1.3,-2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1434,474.0,0.1,6.4,95.0,4.0,1.0,3.0,999.9,4.0,"-1,0",5.9,1.5,-1.0
1435,65.0,0.0,6.4,93.0,10.0,4.0,3.0,983.9,4.0,"-1,0",5.2,3.1,-1.0
1436,159.0,0.1,8.5,83.0,10.0,2.0,1.0,969.7,4.0,"1,0",1.4,1.9,1.0
1437,99.0,0.1,7.5,91.0,10.0,3.0,3.0,995.4,4.0,"-4,0",2.7,1.7,-4.0


In [34]:
# Зависимая переменная для всех одна и та же. Предикторы выбираем случайнм образом.
columns = list(data.columns)
n_x = 5

y_label = ['water_level']
x_labels = random.sample(columns[1:], n_x)

print(x_labels)

['visibility', 'pressure', 'wl_changer', 'temp_change', 'weather']


In [35]:
# Преобразуйте значения всех необходимых параметров к отрезку [0,1].
# Решите получившуюся задачу регрессии с помощью выбранных моделей и сравните их эффективность.


In [36]:
all_lb =  x_labels + y_label

In [37]:

data=data[list(all_lb)]
data

,visibility,pressure,wl_changer,temp_change,weather,water_level
0,100.0,993.8,-3.0,-0.3,0.0,468.0
1,20.0,1004.1,-2.0,0.6,0.0,140.0
2,20.0,1001.2,-7.0,-1.8,1.0,464.0
3,100.0,992.3,-1.0,1.2,0.0,467.0
4,20.0,1001.3,-2.0,2.5,0.0,138.0
...,...,...,...,...,...,...
1434,4.0,999.9,-1.0,5.9,3.0,474.0
1435,10.0,983.9,-1.0,5.2,3.0,65.0
1436,10.0,969.7,1.0,1.4,1.0,159.0
1437,10.0,995.4,-4.0,2.7,3.0,99.0


In [38]:
# Определим размер валидационной и тестовой выборок
val_test_size = round(0.2*len(data))
print(val_test_size)

288


In [39]:
# Создадим обучающую, валидационную и тестовую выборки
random_state = my_seed
train_val, test = train_test_split(data, test_size=val_test_size, random_state=random_state)
train, val = train_test_split(train_val, test_size=val_test_size, random_state=random_state)
print(len(train), len(val), len(test))

863 288 288


In [40]:
# Значения в числовых столбцах преобразуем к отрезку [0,1].
# Для настройки скалировщика используем только обучающую выборку.
columns_to_scale = all_lb

ct = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), columns_to_scale)], remainder='passthrough')
ct.fit(train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['visibility', 'pressure', 'wl_changer',
                                  'temp_change', 'weather', 'water_level'])])

In [41]:
# Преобразуем значения, тип данных приводим к DataFrame
sc_train = pd.DataFrame(ct.transform(train))
sc_test = pd.DataFrame(ct.transform(test))
sc_val = pd.DataFrame(ct.transform(val))

In [42]:
# Устанавливаем названия столбцов
column_names = all_lb
sc_train.columns = column_names
sc_test.columns = column_names
sc_val.columns = column_names

In [43]:
sc_train

,visibility,pressure,wl_changer,temp_change,weather,water_level
0,1.00,0.493284,0.388350,0.410569,0.250,0.321543
1,0.04,0.752239,0.393204,0.504065,0.375,0.499196
2,0.20,0.796269,0.543689,0.422764,0.125,0.557878
3,1.00,0.979104,0.383495,0.430894,0.000,0.381029
4,1.00,0.178358,0.373786,0.341463,0.125,0.197749
...,...,...,...,...,...,...
858,0.20,0.600000,1.000000,0.597561,0.500,0.495177
859,0.20,0.932836,0.393204,0.463415,0.125,0.401929
860,0.10,0.852239,0.388350,0.585366,0.250,0.288585
861,0.20,0.947761,0.393204,0.101626,0.000,0.237138


In [44]:
x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = sc_train[y_label]
y_test = sc_test[y_label]
y_val = sc_val[y_label]

In [45]:
# Обучаем модели
for model in my_models1:
    model.fit(x_train, y_train)

In [46]:
# Оценииваем качество работы моделей на валидационной выборке.
mses = []
for model in my_models1:
    val_pred = model.predict(x_val)
    mse = mean_squared_error(y_val, val_pred)
    mses.append(mse)
    print(mse)

0.033527900160237205
0.03460924665977974
0.032707646348776376
0.0335468200209921


In [47]:
# Выбираем лучшую модель # Укажите, какая модель решает задачу лучше других.
i_min = mses.index(min(mses))
best_my_model = my_models1[i_min]
print('Модель ', best_my_model, 'решает задачу лучше других.') 

Модель  KNeighborsRegressor(n_neighbors=25) решает задачу лучше других.


In [48]:
# Вычислим ошибку лучшей модели на тестовой выборке.
test_pred = best_r_model.predict(x_test)
mse = mean_squared_error(y_test, test_pred)
print(mse)

0.1215092934816489


In [49]:
# Задание №2 - анализ моделей для задачи классификации
# Общий список моделей
c_models = [
    LogisticRegression(penalty='none', solver='saga'),
    LogisticRegression(penalty='l1', solver='saga'),
    LogisticRegression(penalty='l2', solver='saga'),
    LogisticRegression(penalty='elasticnet', l1_ratio=0.25, solver='saga'),
    LogisticRegression(penalty='elasticnet', l1_ratio=0.5, solver='saga'),
    LogisticRegression(penalty='elasticnet', l1_ratio=0.75, solver='saga'),
    LogisticRegression(),
    MultinomialNB(alpha=0.0),
    MultinomialNB(alpha=0.25),
    MultinomialNB(alpha=0.5),
    MultinomialNB(alpha=0.75),
    MultinomialNB(alpha=1.0),
    KNeighborsClassifier(n_neighbors=5),
    KNeighborsClassifier(n_neighbors=10),
    KNeighborsClassifier(n_neighbors=15),
    KNeighborsClassifier(n_neighbors=20),
    KNeighborsClassifier(n_neighbors=25)
]

In [50]:
# Выбор моделей для задания
n = 5
my_models2 = random.sample(c_models, n)
print(my_models2)

[KNeighborsClassifier(n_neighbors=25), MultinomialNB(), MultinomialNB(alpha=0.0), LogisticRegression(), MultinomialNB(alpha=0.25)]


In [51]:
# Загрузим данные для задачи классификации
data = pd.read_csv("datasets/zoo2.csv")
data

,animal_name,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,class_type
0,turtle,0,0,1,0,0,1,0,0,1,1,0,0,4,1,1,1,3
1,chameleon,0,0,1,0,0,0,0,1,1,1,0,0,4,1,1,0,3
2,iguana,0,0,1,0,0,0,1,1,1,1,0,0,4,1,1,1,3
3,lizard,0,0,1,0,0,0,1,1,1,1,0,0,4,1,0,0,3
4,gecko,0,0,1,0,0,0,0,1,1,1,0,0,4,1,1,0,3
5,python,0,0,1,0,0,0,1,1,1,1,1,0,0,1,0,1,3
6,boa,0,0,1,0,0,0,1,1,1,1,0,0,0,1,0,1,3
7,adder,0,0,1,0,0,0,1,1,1,1,1,0,0,1,0,1,3
8,crocodile,0,0,1,0,0,1,1,1,1,1,0,0,4,1,0,1,3
9,alligator,0,0,1,0,0,1,1,1,1,1,0,0,4,1,0,1,3


In [52]:
#data.drop(['animal_name'], axis='columns', inplace=True)

In [53]:
#data.head()

In [54]:
data['animal_name'] = pd.get_dummies(data['animal_name'])
data.head()

,animal_name,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,class_type
0,0,0,0,1,0,0,1,0,0,1,1,0,0,4,1,1,1,3
1,0,0,0,1,0,0,0,0,1,1,1,0,0,4,1,1,0,3
2,0,0,0,1,0,0,0,1,1,1,1,0,0,4,1,1,1,3
3,0,0,0,1,0,0,0,1,1,1,1,0,0,4,1,0,0,3
4,0,0,0,1,0,0,0,0,1,1,1,0,0,4,1,1,0,3


In [55]:
# Метка класса для всех одна и та же. Параметры выбираем случайнм образом.
columns = list(data.columns)
n_x = 8

y_label = ['class_type']
x_labels = random.sample(columns[:-1], n_x)
all_lb1 =  x_labels + y_label
print(x_labels)

['airborne', 'milk', 'eggs', 'fins', 'venomous', 'legs', 'backbone', 'aquatic']


In [56]:
# Преобразуйте значения всех необходимых параметров к отрезку [0,1].
# Решите получившуюся задачу классификации с помощью выбранных моделей и сравните их эффективность.
# Укажите, какая модель решает задачу лучше других.

In [57]:
data=data[list(all_lb1)]
data.head()

,airborne,milk,eggs,fins,venomous,legs,backbone,aquatic,class_type
0,0,0,1,0,0,4,1,1,3
1,0,0,1,0,0,4,1,0,3
2,0,0,1,0,0,4,1,0,3
3,0,0,1,0,0,4,1,0,3
4,0,0,1,0,0,4,1,0,3


In [58]:
# Определим размер валидационной и тестовой выборок
val_test_size = round(0.2*len(data))
print(val_test_size)

9


In [59]:
# Создадим обучающую, валидационную и тестовую выборки
random_state = my_seed
train_val, test = train_test_split(data, test_size=val_test_size, random_state=random_state)
train, val = train_test_split(train_val, test_size=val_test_size, random_state=random_state)
print(len(train), len(val), len(test))

25 9 9


In [60]:
# Значения в числовых столбцах преобразуем к отрезку [0,1].
# Для настройки скалировщика используем только обучающую выборку.
columns_to_scale = x_labels#all_lb1

ct = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), columns_to_scale)], remainder='passthrough')
ct.fit(train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['airborne', 'milk', 'eggs', 'fins',
                                  'venomous', 'legs', 'backbone', 'aquatic'])])

In [61]:
# Преобразуем значения, тип данных приводим к DataFrame
sc_train = pd.DataFrame(ct.transform(train))
sc_test = pd.DataFrame(ct.transform(test))
sc_val = pd.DataFrame(ct.transform(val))

In [62]:
# Устанавливаем названия столбцов
column_names = all_lb1
sc_train.columns = column_names
sc_test.columns = column_names
sc_val.columns = column_names

In [63]:
sc_train

,airborne,milk,eggs,fins,venomous,legs,backbone,aquatic,class_type
0,1.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,6.0
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,7.0
2,0.0,0.0,0.0,1.0,0.0,0.000000,1.0,1.0,4.0
3,0.0,0.0,0.0,1.0,0.0,0.000000,1.0,1.0,4.0
4,1.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,6.0
5,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,0.0,3.0
6,0.0,0.0,0.0,0.0,0.0,0.666667,1.0,1.0,3.0
7,0.0,0.0,0.0,0.0,0.0,0.333333,1.0,1.0,5.0
8,0.0,0.0,0.0,0.0,0.0,0.666667,1.0,0.0,3.0
9,0.0,0.0,0.0,0.0,0.0,0.666667,1.0,1.0,5.0


In [64]:
x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = np.ravel(sc_train[y_label])
y_test = np.ravel(sc_test[y_label])
y_val = np.ravel(sc_val[y_label])

In [65]:
# Обучаем модели
for model2 in my_models2:
    model2.fit(x_train, y_train)

/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/naive_bayes.py:509: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


In [66]:
# Оценииваем качество работы моделей на валидационной выборке.
f1s = []
for model in my_models2:
    val_pred = model.predict(x_val)
    f1 = f1_score(y_val, val_pred, average='weighted')
    f1s.append(f1)
    print(f1)

0.0808080808080808
0.38765432098765434
0.6111111111111112
0.8444444444444444
0.7037037037037037


In [67]:
# Выбираем лучшую модель # Укажите, какая модель решает задачу лучше других.
i_min = f1s.index(min(f1s))
best_my_models2 = my_models2[i_min]
print('Модель ', best_my_models2, 'решает задачу лучше других.') 

Модель  KNeighborsClassifier(n_neighbors=25) решает задачу лучше других.


In [68]:
# Вычислим ошибку лучшей модели на тестовой выборке.
test_pred = best_my_models2.predict(x_test)
f1 = f1_score(y_test, test_pred, average='weighted')
print(f1)

0.02222222222222222
